In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import matplotlib.pyplot as plt
import os
print(os.listdir("../input"))
import tensorflow as tf
from keras.layers import Conv2D,Concatenate,Dense,Add,SpatialDropout2D,SeparableConv2D,Activation,Reshape,Bidirectional,Dropout,Multiply,LeakyReLU,Input,GlobalAveragePooling2D,BatchNormalization,Activation,ZeroPadding1D,Concatenate,Dropout,LSTM
from keras.models import Model,Sequential
from tqdm import tqdm_notebook
from keras.optimizers import Adam
import keras
# Any results you write to the current directory are saved as output.

['train_noisy', 'sample_submission.csv', 'train_curated', 'train_curated.csv', 'train_noisy.csv', 'test']


Using TensorFlow backend.


In [2]:
df=pd.read_csv("../input/train_curated.csv")

In [3]:
df.head()

,fname,labels
0,0006ae4e.wav,Bark
1,0019ef41.wav,Raindrop
2,001ec0ad.wav,Finger_snapping
3,0026c7cb.wav,Run
4,0026f116.wav,Finger_snapping


In [4]:
testdf=pd.read_csv("../input/sample_submission.csv")

In [5]:
label_columns=testdf.columns[1:]
labelmapping=dict((label,index) for index,label in enumerate(label_columns))
for col in label_columns:
    df[col]=0

In [6]:
labelmapping

{'Accelerating_and_revving_and_vroom': 0,
 'Accordion': 1,
 'Acoustic_guitar': 2,
 'Applause': 3,
 'Bark': 4,
 'Bass_drum': 5,
 'Bass_guitar': 6,
 'Bathtub_(filling_or_washing)': 7,
 'Bicycle_bell': 8,
 'Burping_and_eructation': 9,
 'Bus': 10,
 'Buzz': 11,
 'Car_passing_by': 12,
 'Cheering': 13,
 'Chewing_and_mastication': 14,
 'Child_speech_and_kid_speaking': 15,
 'Chink_and_clink': 16,
 'Chirp_and_tweet': 17,
 'Church_bell': 18,
 'Clapping': 19,
 'Computer_keyboard': 20,
 'Crackle': 21,
 'Cricket': 22,
 'Crowd': 23,
 'Cupboard_open_or_close': 24,
 'Cutlery_and_silverware': 25,
 'Dishes_and_pots_and_pans': 26,
 'Drawer_open_or_close': 27,
 'Drip': 28,
 'Electric_guitar': 29,
 'Fart': 30,
 'Female_singing': 31,
 'Female_speech_and_woman_speaking': 32,
 'Fill_(with_liquid)': 33,
 'Finger_snapping': 34,
 'Frying_(food)': 35,
 'Gasp': 36,
 'Glockenspiel': 37,
 'Gong': 38,
 'Gurgling': 39,
 'Harmonica': 40,
 'Hi-hat': 41,
 'Hiss': 42,
 'Keys_jangling': 43,
 'Knock': 44,
 'Male_singing': 45

In [7]:
def fun(x):
    a=np.zeros(80)
    for i in x.split(","):
        a[labelmapping[i]]=1
    return a

In [8]:
def labelling_training(rowlabels,nclasses):
    rows_label_list=[]
    for row in rowlabels:
        p=row.split(",")
        labelsarray=np.zeros((nclasses))
        for label in p:
            index=labelmapping[label]
            labelsarray[index]=1
        rows_label_list.append(labelsarray)
    return rows_label_list

In [9]:
df.head()

,fname,labels,Accelerating_and_revving_and_vroom,Accordion,Acoustic_guitar,Applause,Bark,Bass_drum,Bass_guitar,Bathtub_(filling_or_washing),Bicycle_bell,Burping_and_eructation,Bus,Buzz,Car_passing_by,Cheering,Chewing_and_mastication,Child_speech_and_kid_speaking,Chink_and_clink,Chirp_and_tweet,Church_bell,Clapping,Computer_keyboard,Crackle,Cricket,Crowd,Cupboard_open_or_close,Cutlery_and_silverware,Dishes_and_pots_and_pans,Drawer_open_or_close,Drip,Electric_guitar,Fart,Female_singing,Female_speech_and_woman_speaking,Fill_(with_liquid),Finger_snapping,Frying_(food),Gasp,Glockenspiel,...,Harmonica,Hi-hat,Hiss,Keys_jangling,Knock,Male_singing,Male_speech_and_man_speaking,Marimba_and_xylophone,Mechanical_fan,Meow,Microwave_oven,Motorcycle,Printer,Purr,Race_car_and_auto_racing,Raindrop,Run,Scissors,Screaming,Shatter,Sigh,Sink_(filling_or_washing),Skateboard,Slam,Sneeze,Squeak,Stream,Strum,Tap,Tick-tock,Toilet_flush,Traffic_noise_and_roadway_noise,Trickle_and_dribble,Walk_and_footsteps,Water_tap_and_faucet,Waves_and_surf,Whispering,Writing,Yell,Zipper_(clothing)
0,0006ae4e.wav,Bark,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0019ef41.wav,Raindrop,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,001ec0ad.wav,Finger_snapping,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0026c7cb.wav,Run,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0026f116.wav,Finger_snapping,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
df[label_columns]=labelling_training(df.labels,80)

In [11]:
df['numlabels']=df[label_columns].sum(axis=1)

In [12]:
audiodir1=glob.glob("../input/train_curated/*.wav")
audiodir2=glob.glob("../input/train_noisy/*.wav")

In [13]:
print(len(audiodir1))
print(len(audiodir2))

4970
19815


In [14]:
import librosa

In [15]:
sr=26000
duration=9
hop_length=400
fmin=20
fmax=sr//2
n_mels=128
n_fft=n_mels*20
samples=sr*duration

In [16]:
def read_audio(path):
    '''
    Reads in the audio file and returns
    an array that we can turn into a melspectogram
    '''
    y, _ = librosa.core.load(path, sr=44100)
    # trim silence
    if 0 < len(y): # workaround: 0 length causes error
        y, _ = librosa.effects.trim(y)
    if len(y) > samples: # long enough
        y = y[0:0+samples]
    else: # pad blank
        padding = samples - len(y)
        offset = padding // 2
        y = np.pad(y, (offset, samples - len(y) - offset), 'constant')
    return y

def audio_to_melspectrogram(audio):
    '''
    Convert to melspectrogram after audio is read in
    '''
    spectrogram = librosa.feature.melspectrogram(audio, 
                                                 sr=sr,
                                                 n_mels=n_mels,
                                                 hop_length=hop_length,
                                                 n_fft=n_fft,
                                                 fmin=fmin,
                                                 fmax=fmax)
    return librosa.power_to_db(spectrogram).astype(np.float32)

def read_as_melspectrogram(path):
    '''
    Convert audio into a melspectrogram 
    so we can use machine learning
    '''
    mels = audio_to_melspectrogram(read_audio(path))
    return mels

def convert_wav_to_image(df, path):
    X = []
    for _,row in tqdm_notebook(df.iterrows()):
        x = read_as_melspectrogram('{}/{}'.format(path[0],
                                                  str(row['fname'])))
        X.append(x.transpose())
    return X

def normalize(img):
    '''
    Normalizes an array 
    (subtract mean and divide by standard deviation)
    '''
    eps = 0.001
    if np.std(img) != 0:
        img = (img - np.mean(img)) / np.std(img)
    else:
        img = (img - np.mean(img)) / eps
    return img

def normalize_dataset(X):
    '''
    Normalizes list of arrays
    (subtract mean and divide by standard deviation)
    '''
    normalized_dataset = []
    for img in X:
        normalized = normalize(img)
        normalized_dataset.append(normalized)
    return normalized_dataset

In [17]:
KAGGLE_DIR = '../input/'
train_curated_path = KAGGLE_DIR + 'train_curated/'
test_path = KAGGLE_DIR + 'test/'

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X = np.array(convert_wav_to_image(df, [train_curated_path]))
X = normalize_dataset(X)
Y = df[label_columns].values

In [20]:
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.1, random_state=42)

In [21]:
y_train.shape

(4473, 80)

In [22]:
x_train[0].shape

(586, 128)

In [23]:
x_train=np.array(x_train)
x_val=np.array(x_val)

In [24]:
x_train=x_train.astype('float16')
x_val=x_val.astype('float16')

In [25]:
X=np.array(1)
Y=np.array(1)

In [26]:
x_train=x_train.reshape((-1,586,128,1))

In [27]:
x_train.shape

(4473, 586, 128, 1)

In [28]:
input_shape=(586,128,1)
optimizer=Adam(0.008,beta_1=1.0,beta_2=0.001,amsgrad=True)
n_classes=80

In [29]:
params_conv2d = {
        "padding": "SAME",
        "activation": keras.activations.elu
        # 'kernel_regularizer': keras.regularizers.l2(0.01)
}


In [30]:
def spatial_squeeze_excite_block(input1):
    se=Conv2D(1,(1,1),activation='sigmoid',kernel_initializer='he_normal')(input1)
    x=Multiply()([input1,se])
    return x

def squeeze_excite_block(input1,ratio=4):
    se=GlobalAveragePooling2D()(input1)
    se=Dense(input1._keras_shape[-1]//ratio,activation='relu')(se)
    se=Dense(input1._keras_shape[-1],activation='sigmoid')(se)
    se=Reshape((1,1,input1._keras_shape[-1]))(se)
    finput=Multiply()([input1,se])
    return finput

def spatial_Csqueeze_excite_block(input1,ratio=4):
    squeezeblock=squeeze_excite_block(input1,ratio)
    spatialblock=spatial_squeeze_excite_block(input1)
    finalblock=Add()([squeezeblock,spatialblock])
    return finalblock

In [31]:
inp = keras.Input((586,128,1))

# conv_1
conv1 = Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(inp)
conv1 = keras.layers.SpatialDropout2D(.2, data_format='channels_last')(conv1)
conv1 = BatchNormalization()(conv1)
conv1 = Activation('relu')(conv1)
conv1 = spatial_Csqueeze_excite_block(conv1)
conv2 = Conv2D(filters=32, kernel_size=4, strides=1, padding='same')(inp)
conv2 = keras.layers.SpatialDropout2D(.2, data_format='channels_last')(conv2)
conv2 = BatchNormalization()(conv2)
conv2 = Activation('relu')(conv2)
conv2 = spatial_Csqueeze_excite_block(conv2)
conv3 = Conv2D(filters=32, kernel_size=5, strides=1, padding='same')(inp)
conv3 = keras.layers.SpatialDropout2D(.2, data_format='channels_last')(conv3)
cconv3 = BatchNormalization()(conv3)
conv3 = Activation('relu')(conv3)
conv3 = spatial_Csqueeze_excite_block(conv3)
concat= Concatenate(axis=-1)([conv1,conv2])
concat = spatial_Csqueeze_excite_block(concat)

#conv31 = Conv2D(64,(1,1))(concat)
#conv31 = BatchNormalization()(conv31)
#conv31= Activation('relu')(conv31)
conv31 = Conv2D(32,kernel_size=3,strides=1,padding='same')(concat)
#conv31= SpatialDropout2D(0.2,data_format='channels_last')(conv31)
conv31 = BatchNormalization()(conv31)
conv31 = Activation('relu')(conv31)
conv31= spatial_Csqueeze_excite_block(conv31)
max_pool31 = keras.layers.MaxPool2D(padding="SAME")(conv31)

##conv32 = Conv2D(64,(1,1))(concat)
#conv32 = BatchNormalization()(conv32)
#conv32= Activation('relu')(conv32)
conv32= Conv2D(32,kernel_size=4,strides=1,padding='same')(concat)
#conv32= SpatialDropout2D(0.2,data_format='channels_last')(conv32)
conv32 = BatchNormalization()(conv32)
conv32 = Activation('relu')(conv32)
conv32=spatial_Csqueeze_excite_block(conv32)
max_pool32 = keras.layers.MaxPool2D(padding="SAME")(conv32)

concat2=Concatenate()([max_pool31,max_pool32])
concat2=spatial_Csqueeze_excite_block(concat2)

conv41= SeparableConv2D(128,kernel_size=3,strides=1,padding='same')(concat2)
#conv41= SpatialDropout2D(0.4,data_format='channels_last')(conv41)
conv41 = BatchNormalization()(conv41)
conv41 = Activation('relu')(conv41)
#conv41=spatial_Csqueeze_excite_block(conv41)
max_pool41 = keras.layers.MaxPool2D(padding="SAME")(conv41)

conv51=SeparableConv2D(256,kernel_size=4,strides=1,padding='same')(concat2)
#conv51= SpatialDropout2D(0.4,data_format='channels_last')(conv51)
conv51 = BatchNormalization()(conv51)
conv51 = Activation('relu')(conv51)
conv51=spatial_Csqueeze_excite_block(conv51)
max_pool51 = keras.layers.MaxPool2D(padding="SAME")(conv51)
concat4=Concatenate()([max_pool51,max_pool41])

avg_pool = GlobalAveragePooling2D()(concat4)
dense10 = keras.layers.Dense(80,activation='sigmoid')(avg_pool)


model = keras.Model(inputs=inp,outputs=dense10)

In [32]:
dense10.shape[-1]

Dimension(80)

In [33]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 586, 128, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 586, 128, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 586, 128, 32) 544         input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout2d_1 (SpatialDro (None, 586, 128, 32) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
spatial_dr

In [34]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [35]:
history=model.fit(x_train,y_train,batch_size=24,epochs=26,verbose=1,shuffle=True,validation_split=0.05)

Train on 4249 samples, validate on 224 samples
Epoch 1/26
4249/4249 [==============================] - 120s 28ms/step - loss: 0.1009 - acc: 0.9754 - val_loss: 0.0710 - val_acc: 0.9856
Epoch 2/26
4249/4249 [==============================] - 113s 27ms/step - loss: 0.0687 - acc: 0.9855 - val_loss: 0.0680 - val_acc: 0.9857
Epoch 3/26
4249/4249 [==============================] - 113s 27ms/step - loss: 0.0657 - acc: 0.9855 - val_loss: 0.0651 - val_acc: 0.9858
Epoch 4/26
4249/4249 [==============================] - 113s 27ms/step - loss: 0.0638 - acc: 0.9857 - val_loss: 0.0632 - val_acc: 0.9858
Epoch 5/26
4249/4249 [==============================] - 113s 27ms/step - loss: 0.0617 - acc: 0.9857 - val_loss: 0.0621 - val_acc: 0.9857
Epoch 6/26
4249/4249 [==============================] - 113s 27ms/step - loss: 0.0603 - acc: 0.9857 - val_loss: 0.0616 - val_acc: 0.9858
Epoch 7/26
4249/4249 [==============================] - 113s 27ms/step - loss: 0.0592 - acc: 0.9857 - val_loss: 0.0617 - val_acc: 0

In [36]:
#Important Concept learnt while training:- Too much dropout hurt models performance. Dropout rate is a hyperparameter

In [37]:
x_train=0
y_train=0

In [38]:
X_test = np.array(convert_wav_to_image(testdf, [test_path]))
X_test = normalize_dataset(X_test)

In [39]:
X_test=np.array(X_test).reshape((-1,586,128,1))

In [40]:
predictions = model.predict(X_test)
# Save predictions in a csv file
testdf[label_columns] = predictions
testdf.to_csv('submission.csv', index=False)